In [12]:
import sqlite3
import pandas as pd
import plotly
from plotly import figure_factory as FF
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [3]:
try:
    db_conn = sqlite3.connect('./db/ib_api.db')
except Exception as e:
    print(e)
    raise e

In [4]:
query = """select datetime(a.date, 'localtime') insert_date, b.symbol, a.ticktype, a.value
    from tick_market_data a join tickers b 
    on a.tickerid = b.requestid
    where b.requestid = 1
    and a.date BETWEEN '2019-08-15'  and '2019-08-16'
    and a.ticktype = 'DELAYED_ASK'
    """

In [5]:
ticker_data = pd.read_sql_query(sql=query, con=db_conn)

In [6]:
ticker_data.insert_date = pd.to_datetime(ticker_data.insert_date)

In [7]:
ticker_data.set_index('insert_date', inplace=True)

In [8]:
ohlc_5mins = ticker_data['value'].resample('5Min').ohlc()

In [9]:
ohlc_5mins.head()

,open,high,low,close
insert_date,,,,
2019-08-15 12:25:00,36.42,36.48,36.33,36.34
2019-08-15 12:30:00,36.33,36.58,36.32,36.58
2019-08-15 12:35:00,36.59,36.65,36.53,36.65
2019-08-15 12:40:00,36.64,36.64,36.44,36.58
2019-08-15 12:45:00,36.54,36.62,36.50,36.55


In [10]:
fig = FF.create_ohlc(ohlc_5mins.open, ohlc_5mins.high, ohlc_5mins.low, ohlc_5mins.close, dates=ohlc_5mins.index)

In [11]:
plotly.offline.iplot(fig, filename='sample_ohlc')